In [1]:
import tensorflow as tf
from data import preproc as pp

In [1]:
import os

EPOCHS = 1000
BATCH_SIZE = 16
ARCH = "flor"

IMG_SIZE = (512, 128, 1)
DATA_ROOT_PATH = "C:\\Users\\pavan\\Documents\\4 th yr\\btechp\\ocr\\data\\"
IMAGES_PATH = os.path.join(DATA_ROOT_PATH, "images", "data")
IMAGES_PATH

'C:\\Users\\pavan\\Documents\\4 th yr\\btechp\\ocr\\data\\images\\data'

In [7]:
from PIL import Image, ImageOps
maxh=0
maxw=0
for i in os.listdir(IMAGES_PATH):
    l=os.listdir(IMAGES_PATH+"/"+i)
    l.pop()
    print(i,"->",end=" ")
    for j in l[0:10]:
        for k in os.listdir(IMAGES_PATH+"/"+i+"/"+j):
            
            if "jpg" in k:
                ime=Image.open(IMAGES_PATH+"/"+i+"/"+j+"/"+k)
                maxh=max(maxh,ime.height)
                maxw=max(maxw,ime.width)
        print(j,end=" ")
    print()
        
        

print(maxh,maxw)

1 -> 1 10 100 101 102 103 104 105 106 107 
10 -> 1 10 100 101 102 103 104 105 106 107 
11 -> 1 10 11 12 13 14 15 16 17 18 
12 -> 1 10 11 12 13 14 15 16 17 18 
2 -> 1 10 100 101 102 103 104 105 106 107 
3 -> 1 10 100 101 102 103 104 105 106 107 
4 -> 1 10 100 101 102 103 104 105 106 107 
5 -> 1 10 100 101 102 103 104 105 106 107 
6 -> 1 10 100 101 102 103 104 105 106 107 
7 -> 1 10 100 101 102 103 104 105 106 107 
8 -> 1 10 100 101 102 103 104 105 106 107 
9 -> 1 10 100 101 102 103 104 105 106 107 
1642 6975


In [5]:
vocab = [str(vocab_to_idx[item.strip()]) for item in vocab]

NameError: name 'vocab' is not defined

In [96]:
new_vocab = "\n".join(vocab)

with open("new_voc.txt","w") as f:
  f.write(new_vocab)

In [7]:
gvocab = None

with open(os.path.join(DATA_ROOT_PATH, "hindi_vocab.txt")) as f:
  vocab = f.readlines()

idx_to_vocab = {i:value.strip() for i, value in enumerate(vocab)}
vocab_to_idx = {value:key for key, value in idx_to_vocab.items()}

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 11: character maps to <undefined>

In [9]:
train_data = None

with open("C:\\Users\\pavan\\Documents\\4 th yr\\btechp\\ocr\\data\\test.txt") as f:
  train_data = f.readlines()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 39: character maps to <undefined>

In [78]:
train_data[0].split()

['HindiSeg/train/10/259/1.jpg', 'हांगकांग']

In [10]:
train_data_new = ""

for item in train_data:
  path, label = item.split()
  label = vocab_to_idx[label]
  line = " ".join([path, str(label)])
  line +="\n"
  train_data_new += line

TypeError: 'NoneType' object is not iterable

In [11]:
with open("new_test.txt", "w") as f:
  f.write(train_data_new)

In [12]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import tensorflow as tf

class DataGen(tf.keras.utils.Sequence):
    def __init__(self, data, img_size, batch_size, mode="TRAIN"):
        self.data = data
        self.img_size = img_size
        self.batch_size = batch_size
        self.mode = mode
        
    def __getitem__(self, i):
        start = i * self.batch_size
        end = (i+1) * self.batch_size
        
        batch_images = np.zeros((self.batch_size, self.img_size[0], self.img_size[1], 1))
        batch_labels = np.zeros((self.batch_size, 1))
        
        for ii, df_index in enumerate(range(start, end)):
            curr_data = self.data[ii].split()
            curr_img_path = curr_data[0]
            curr_label = vocab_to_idx[curr_data[1]]

            curr_img_path = "/".join(curr_img_path.split("/")[2:])
            curr_img_path = os.path.join(IMAGES_PATH, curr_img_path)
            
            curr_img = img_to_array(load_img(curr_img_path, color_mode="grayscale"))
            img_shape = curr_img.shape
            
            curr_img = tf.image.resize(curr_img, (self.img_size[0], self.img_size[1]), method="nearest")
            curr_img = curr_img.numpy().reshape((self.img_size[0], self.img_size[1]))
            curr_img = pp.preprocess(curr_img, self.img_size)
            batch_images[ii, :, :, 0] = curr_img / 255.
            batch_labels[ii, :] = curr_label
                    
        if self.mode == "TRAIN":
          return batch_images, batch_labels
        else:
          return batch_images
        
    def __len__(self):
      return len(self.data) // self.batch_size

In [103]:
!cp ocr.zip ./drive/MyDrive/Project/